In [1]:
%matplotlib inline
import xarray as xr
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import os 
import gcsfs

import xesmf as xe
from time import gmtime, strftime

In [2]:
fs = gcsfs.GCSFileSystem(token='/opt/gcsfuse_tokens/impactlab-data.json')

load example domain file (from RASM, but it's essentially the same as CESM - RASM has the same structure as CESM, just different component models)

In [3]:
domain_rasm = xr.open_dataset('./domain.lnd.wr50a_ar9v4.100920.nc')

define domain grids using `xesmf`

In [5]:
def create_domain_dataset(resolution=1, lat_name='lat', lon_name='lon'):
    grid_global = xe.util.grid_global(resolution, resolution)
    domain = grid_global.rename({"x": lon_name, "y": lat_name})

    domain[lat_name] = np.unique(domain[lat_name].values)
    domain[lon_name] = np.unique(domain[lon_name].values)
    domain['lon_b'] = np.unique(domain['lon_b'].values)
    domain['lat_b'] = np.unique(domain['lat_b'].values)
    
    attrs = {'title': 'Rhodium/CIL bias correction and downscaling %s-degree grid' %str(resolution), 
         'history': 'created by Diana Gergel, %s' %strftime("%Y-%m-%d %H:%M:%S", gmtime()), 
         'source code': 'grid specified by xesmf, xe.util.grid_global(%s, %s)' %(resolution, resolution)}
    domain.attrs.update(attrs)
    domain['lon'].attrs.update({'long_name': 'longitude of grid cell center', 'units': 'degrees_east'})
    domain['lat'].attrs.update({'long_name': 'latitude of grid cell center', 'units': 'degrees_north'})
    domain['lon_b'].attrs.update({'long_name': 'longitude bounds', 'units': 'degrees_east'})
    domain['lat_b'].attrs.update({'long_name': 'latitude bounds', 'units': 'degrees_north'})
    
    return domain 

In [6]:
domain_coarse = create_domain_dataset(resolution=1)

In [7]:
domain_fine = create_domain_dataset(resolution=0.25)

save as NetCDFs and zarrs 

In [8]:
domain_coarse_filename = '/home/jovyan/downscaling/downscale/domain.1x1.nc'
domain_fine_filename = '/home/jovyan/downscaling/downscale/domain.0p25x0p25.nc'

In [10]:
'''coarse_zarr = 'gs://impactlab-data/climate/downscaling/domain.1x1.zarr'
fine_zarr = 'gs://impactlab-data/climate/downscaling/domain.0p25x0p25.zarr'
coarse_store = fs.get_mapper(coarse_zarr, check=False)
fine_store = fs.get_mapper(fine_zarr, check=False)'''

In [13]:
domain_coarse.to_netcdf(domain_coarse_filename)
domain_fine.to_netcdf(domain_fine_filename)

'''domain_coarse.to_zarr(coarse_zarr, consolidated=True, mode="w")
domain_fine.to_zarr(fine_zarr, consolidated=True, mode="w")'''

'domain_coarse.to_zarr(coarse_zarr, consolidated=True, mode="w")\ndomain_fine.to_zarr(fine_zarr, consolidated=True, mode="w")'

test regridding and saved domain file to ensure that regridder service updates in dodola will address rechunking service issues 

In [14]:
# regrid an ERA-5 file to the domain file grid 
era5 = xr.open_dataset(os.path.join('/gcs/impactlab-data/climate/source_data/ERA-5/day/tmax/v1.1', 'tmax_daily_2000-2000.nc'))

In [18]:
def validate_domain_file(filepath, test_ds, zarr=False):
    if zarr:
        domain = xr.open_zarr(filepath)
    else: 
        domain = xr.open_dataset(filepath)
        
    regridder = xe.Regridder(test_ds.rename({'latitude': 'lat', 'longitude': 'lon'}), domain, method='bilinear')
    regridded_ds = regridder(test_ds['tmax'])
    
    return regridded_ds

In [19]:
era5_regridded = validate_domain_file(domain_fine_filename, era5)

/srv/conda/envs/notebook/lib/python3.8/site-packages/xesmf/frontend.py:450: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  dr_out = xr.apply_ufunc(
